### Import libraries

In [1]:
import pandas as pd
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error

### Set the data repository as the working directory 

In [2]:
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\codes


In [3]:
os.chdir("../data")
print(os.path.abspath(os.curdir))

C:\Users\Asus\Documents\College\UP - MS EE\SY 2022-2023 Second Semester (temporary, final files in new laptop)\EE 290\Files (NEW Jun 22) FINAL FINAL\data


### Import the test set (solar and wind inclusion)

##### Test set (solar and wind inclusion)

In [4]:
pd_qd_test_solar_wind_15_pct = pd.read_csv('pd_qd_test_solar_wind_15_pct.csv')
pd_qd_test_solar_wind_15_pct.head()

,scenario,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,1,0.034699,0.007573,0.045639,0.018038,0.053473,0.010757,0.031436,0.009529,0.027359,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,2,0.034556,0.007371,0.045064,0.017669,0.052019,0.010745,0.030837,0.009231,0.026833,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,3,0.034414,0.007166,0.044490,0.017302,0.049703,0.009853,0.030238,0.009051,0.026307,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,4,0.034272,0.006959,0.043915,0.017079,0.048373,0.009562,0.029639,0.008872,0.025781,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,5,0.034130,0.006750,0.043340,0.016992,0.047043,0.008971,0.029040,0.008693,0.025255,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [5]:
pg_v_test_solar_wind_15_pct = pd.read_csv('pg_v_test_solar_wind_15_pct.csv')
pg_v_test_solar_wind_15_pct.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,5.145971e-07,0.363358,1.023155,0.444721,1.005264,0.499989,1.001171,0.499986,1.016907
1,2,2.322973e-06,0.351056,1.023037,0.425181,1.004975,0.499894,1.001296,0.500000,1.017366
2,3,3.246355e-07,0.337529,1.022865,0.404762,1.004671,0.499991,1.001440,0.499986,1.017816
3,4,8.400630e-07,0.325118,1.022749,0.385618,1.004388,0.499958,1.001578,0.500000,1.018277
4,5,3.752829e-07,0.312718,1.022616,0.366442,1.004110,0.499976,1.001719,0.500000,1.018735


### Separate the dependent variables (optimal generator setpoints) and the independent variables (non-dispatchable loads)
Non-dispatchable loads: $x = (P_{D_{i}},Q_{D_{i}})\forall i\in \Omega_{D}$
<br>
Optimal generator setpoints: $y = (P_{G_{j}},V_{j})\forall j\in \Omega_{G}$

##### Test set (solar and wind inclusion)

In [6]:
x_test_solar_wind_15_pct = pd_qd_test_solar_wind_15_pct.loc[:, pd_qd_test_solar_wind_15_pct.columns != 'scenario']
x_test_solar_wind_15_pct.head()

,pd_2,qd_2,pd_3,qd_3,pd_4,qd_4,pd_5,qd_5,pd_6,qd_6,...,pd_29,qd_29,pd_30,qd_30,pd_31,qd_31,pd_32,qd_32,pd_33,qd_33
0,0.034699,0.007573,0.045639,0.018038,0.053473,0.010757,0.031436,0.009529,0.027359,0.019239,...,0.063776,0.040361,0.101603,0.058396,0.077677,0.030700,0.117351,0.086486,0.024336,0.002183
1,0.034556,0.007371,0.045064,0.017669,0.052019,0.010745,0.030837,0.009231,0.026833,0.018800,...,0.062670,0.039661,0.099900,0.057418,0.076220,0.029884,0.115035,0.085078,0.024143,0.002166
2,0.034414,0.007166,0.044490,0.017302,0.049703,0.009853,0.030238,0.009051,0.026307,0.018499,...,0.061563,0.039122,0.098197,0.056177,0.074763,0.029548,0.112719,0.083659,0.023950,0.002149
3,0.034272,0.006959,0.043915,0.017079,0.048373,0.009562,0.029639,0.008872,0.025781,0.018130,...,0.060457,0.038577,0.096495,0.055202,0.073306,0.028741,0.110403,0.082227,0.023757,0.002131
4,0.034130,0.006750,0.043340,0.016992,0.047043,0.008971,0.029040,0.008693,0.025255,0.017825,...,0.059350,0.037716,0.094792,0.054482,0.071848,0.028396,0.108087,0.080502,0.023564,0.002114


In [7]:
y_test_solar_wind_15_pct = pg_v_test_solar_wind_15_pct.loc[:, pg_v_test_solar_wind_15_pct.columns != 'scenario']
y_test_solar_wind_15_pct.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,5.145971e-07,0.363358,1.023155,0.444721,1.005264,0.499989,1.001171,0.499986,1.016907
1,2.322973e-06,0.351056,1.023037,0.425181,1.004975,0.499894,1.001296,0.500000,1.017366
2,3.246355e-07,0.337529,1.022865,0.404762,1.004671,0.499991,1.001440,0.499986,1.017816
3,8.400630e-07,0.325118,1.022749,0.385618,1.004388,0.499958,1.001578,0.500000,1.018277
4,3.752829e-07,0.312718,1.022616,0.366442,1.004110,0.499976,1.001719,0.500000,1.018735


### Test the trained models on the test set (solar and wind inclusion)

In [8]:
model_1 = load_model('model_1.h5')
model_2 = load_model('model_2.h5')
model_3 = load_model('model_3.h5')

##### Using model 1 that was trained using training group 1

In [9]:
y_test_solar_wind_15_pct_pred_1 = model_1.predict(x_test_solar_wind_15_pct)
y_test_solar_wind_15_pct_pred_1 = pd.DataFrame(data = y_test_solar_wind_15_pct_pred_1,
                                 columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_15_pct_pred_1.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000113,0.360193,1.023275,0.446233,1.005158,0.499973,1.001090,0.499953,1.017292
1,0.000107,0.347575,1.023141,0.426885,1.004870,0.499980,1.001214,0.499950,1.017754
2,0.000112,0.333954,1.022992,0.406484,1.004555,0.499989,1.001350,0.499947,1.018237
3,0.000110,0.321537,1.022876,0.387352,1.004272,0.499996,1.001490,0.499944,1.018696
4,0.000110,0.309225,1.022746,0.368394,1.003999,0.500004,1.001630,0.499652,1.019140


In [10]:
pg_v_test_solar_wind_15_pct_pred_1 = pd.concat([pg_v_test_solar_wind_15_pct.scenario, y_test_solar_wind_15_pct_pred_1], axis = 1)
pg_v_test_solar_wind_15_pct_pred_1.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000113,0.360193,1.023275,0.446233,1.005158,0.499973,1.001090,0.499953,1.017292
1,2,0.000107,0.347575,1.023141,0.426885,1.004870,0.499980,1.001214,0.499950,1.017754
2,3,0.000112,0.333954,1.022992,0.406484,1.004555,0.499989,1.001350,0.499947,1.018237
3,4,0.000110,0.321537,1.022876,0.387352,1.004272,0.499996,1.001490,0.499944,1.018696
4,5,0.000110,0.309225,1.022746,0.368394,1.003999,0.500004,1.001630,0.499652,1.019140


In [11]:
pg_v_test_solar_wind_15_pct_pred_1.to_csv('pg_v_test_solar_wind_15_pct_pred_1.csv', index = False, float_format = '%0.20f')

##### Using model 2 that was trained using training group 2

In [12]:
y_test_solar_wind_15_pct_pred_2 = model_2.predict(x_test_solar_wind_15_pct)
y_test_solar_wind_15_pct_pred_2 = pd.DataFrame(data = y_test_solar_wind_15_pct_pred_2,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_15_pct_pred_2.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,-0.000016,0.363794,1.022959,0.445074,1.005289,0.499874,1.001148,0.499779,1.016752
1,-0.000010,0.351273,1.022818,0.425658,1.005011,0.499853,1.001267,0.499772,1.017202
2,-0.000003,0.337971,1.022651,0.405154,1.004714,0.499846,1.001403,0.499771,1.017643
3,0.000003,0.325561,1.022525,0.386024,1.004442,0.499841,1.001534,0.499771,1.018092
4,0.000008,0.313151,1.022383,0.366876,1.004174,0.499835,1.001666,0.499771,1.018541


In [13]:
pg_v_test_solar_wind_15_pct_pred_2 = pd.concat([pg_v_test_solar_wind_15_pct.scenario, y_test_solar_wind_15_pct_pred_2], axis = 1)
pg_v_test_solar_wind_15_pct_pred_2.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,-0.000016,0.363794,1.022959,0.445074,1.005289,0.499874,1.001148,0.499779,1.016752
1,2,-0.000010,0.351273,1.022818,0.425658,1.005011,0.499853,1.001267,0.499772,1.017202
2,3,-0.000003,0.337971,1.022651,0.405154,1.004714,0.499846,1.001403,0.499771,1.017643
3,4,0.000003,0.325561,1.022525,0.386024,1.004442,0.499841,1.001534,0.499771,1.018092
4,5,0.000008,0.313151,1.022383,0.366876,1.004174,0.499835,1.001666,0.499771,1.018541


In [14]:
pg_v_test_solar_wind_15_pct_pred_2.to_csv('pg_v_test_solar_wind_15_pct_pred_2.csv', index = False, float_format = '%0.20f')

##### Using model 3 that was trained using training group 3

In [15]:
y_test_solar_wind_15_pct_pred_3 = model_3.predict(x_test_solar_wind_15_pct)
y_test_solar_wind_15_pct_pred_3 = pd.DataFrame(data = y_test_solar_wind_15_pct_pred_3,
                             columns = ['pg_1', 'pg_18', 'v_18', 'pg_22', 'v_22', 'pg_25', 'v_25', 'pg_33', 'v_33'])
y_test_solar_wind_15_pct_pred_3.head()

,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,0.000405,0.363140,1.023209,0.444396,1.005283,0.500095,1.001179,0.500265,1.016931
1,0.000421,0.350521,1.023078,0.425102,1.005000,0.500102,1.001308,0.500276,1.017376
2,0.000438,0.337118,1.022927,0.404628,1.004696,0.500109,1.001455,0.500286,1.017820
3,0.000454,0.324663,1.022817,0.385466,1.004411,0.500116,1.001598,0.500274,1.018284
4,0.000467,0.312220,1.022688,0.366362,1.004132,0.500122,1.001742,0.500276,1.018746


In [16]:
pg_v_test_solar_wind_15_pct_pred_3 = pd.concat([pg_v_test_solar_wind_15_pct.scenario, y_test_solar_wind_15_pct_pred_3], axis = 1)
pg_v_test_solar_wind_15_pct_pred_3.head()

,scenario,pg_1,pg_18,v_18,pg_22,v_22,pg_25,v_25,pg_33,v_33
0,1,0.000405,0.363140,1.023209,0.444396,1.005283,0.500095,1.001179,0.500265,1.016931
1,2,0.000421,0.350521,1.023078,0.425102,1.005000,0.500102,1.001308,0.500276,1.017376
2,3,0.000438,0.337118,1.022927,0.404628,1.004696,0.500109,1.001455,0.500286,1.017820
3,4,0.000454,0.324663,1.022817,0.385466,1.004411,0.500116,1.001598,0.500274,1.018284
4,5,0.000467,0.312220,1.022688,0.366362,1.004132,0.500122,1.001742,0.500276,1.018746


In [17]:
pg_v_test_solar_wind_15_pct_pred_3.to_csv('pg_v_test_solar_wind_15_pct_pred_3.csv', index = False, float_format = '%0.20f')

##### RMSE of $P_{G_{1}}$ estimates

In [18]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_1, pg_v_test_solar_wind_15_pct_pred_1.pg_1, squared = False)

0.00143299653343441

In [19]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_1, pg_v_test_solar_wind_15_pct_pred_2.pg_1, squared = False)

0.002676526406431432

In [20]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_1, pg_v_test_solar_wind_15_pct_pred_3.pg_1, squared = False)

0.05393053219988027

##### RMSE of $P_{G_{18}}$ estimates

In [21]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_18, pg_v_test_solar_wind_15_pct_pred_1.pg_18, squared = False)

0.0007023981970013766

In [22]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_18, pg_v_test_solar_wind_15_pct_pred_2.pg_18, squared = False)

0.0031354238923699583

In [23]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_18, pg_v_test_solar_wind_15_pct_pred_3.pg_18, squared = False)

0.0030167006060402256

##### RMSE of $P_{G_{22}}$ estimates

In [24]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_22, pg_v_test_solar_wind_15_pct_pred_1.pg_22, squared = False)

0.0006572915095265403

In [25]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_22, pg_v_test_solar_wind_15_pct_pred_2.pg_22, squared = False)

0.00047489678224313365

In [26]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_22, pg_v_test_solar_wind_15_pct_pred_3.pg_22, squared = False)

0.0033223615027513395

##### RMSE of $P_{G_{25}}$ estimates

In [27]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_25, pg_v_test_solar_wind_15_pct_pred_1.pg_25, squared = False)

5.991286628163739e-05

In [28]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_25, pg_v_test_solar_wind_15_pct_pred_2.pg_25, squared = False)

0.00018194274897120056

In [29]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_25, pg_v_test_solar_wind_15_pct_pred_3.pg_25, squared = False)

0.006897146990635835

##### RMSE of $P_{G_{33}}$ estimates

In [30]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_33, pg_v_test_solar_wind_15_pct_pred_1.pg_33, squared = False)

0.0005026516691341419

In [31]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_33, pg_v_test_solar_wind_15_pct_pred_2.pg_33, squared = False)

0.0002961904085474518

In [32]:
mean_squared_error(pg_v_test_solar_wind_15_pct.pg_33, pg_v_test_solar_wind_15_pct_pred_3.pg_33, squared = False)

0.0060069219473859285

##### RMSE of $V_{18}$ estimates

In [33]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_18, pg_v_test_solar_wind_15_pct_pred_1.v_18, squared = False)

0.0003016733400972307

In [34]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_18, pg_v_test_solar_wind_15_pct_pred_2.v_18, squared = False)

0.0006054577391978772

In [35]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_18, pg_v_test_solar_wind_15_pct_pred_3.v_18, squared = False)

0.00734216190467019

##### RMSE of $V_{22}$ estimates

In [36]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_22, pg_v_test_solar_wind_15_pct_pred_1.v_22, squared = False)

0.00012710364864682537

In [37]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_22, pg_v_test_solar_wind_15_pct_pred_2.v_22, squared = False)

0.0001862744656925585

In [38]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_22, pg_v_test_solar_wind_15_pct_pred_3.v_22, squared = False)

0.003134476561846133

##### RMSE of $V_{25}$ estimates

In [39]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_25, pg_v_test_solar_wind_15_pct_pred_1.v_25, squared = False)

0.00012232003410856657

In [40]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_25, pg_v_test_solar_wind_15_pct_pred_2.v_25, squared = False)

0.0005876320228214633

In [41]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_25, pg_v_test_solar_wind_15_pct_pred_3.v_25, squared = False)

0.007936493782197674

##### RMSE of $V_{33}$ estimates

In [42]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_33, pg_v_test_solar_wind_15_pct_pred_1.v_33, squared = False)

0.00024607546901610406

In [43]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_33, pg_v_test_solar_wind_15_pct_pred_2.v_33, squared = False)

0.00020836463394501234

In [44]:
mean_squared_error(pg_v_test_solar_wind_15_pct.v_33, pg_v_test_solar_wind_15_pct_pred_3.v_33, squared = False)

0.0015144423135035575